# Code for FedEraser

Adapted from original code of [the FedEraser paper](https://ieeexplore.ieee.org/abstract/document/9521274).

Create a virtual environment using conda by

```shell
conda create -n federaser python=3.8.3
```

and activating it

```shell
conda activate federaser
```

Then, install required packages

```
# This file may be used to create an environment using:
# $ conda create --name <env> --file <this file>
# platform: linux-64
_libgcc_mutex=0.1=main
_openmp_mutex=5.1=1_gnu
_py-xgboost-mutex=2.0=cpu_0
blas=1.0=mkl
ca-certificates=2022.10.11=h06a4308_0
certifi=2022.9.24=py38h06a4308_0
cudatoolkit=10.2.89=hfd86e86_1
freetype=2.12.1=h4a9f257_0
giflib=5.2.1=h7b6447c_0
intel-openmp=2021.4.0=h06a4308_3561
joblib=1.1.1=py38h06a4308_0
jpeg=9e=h7f8727e_0
lcms2=2.12=h3be6417_0
ld_impl_linux-64=2.38=h1181459_1
lerc=3.0=h295c915_0
libdeflate=1.8=h7f8727e_5
libffi=3.3=he6710b0_2
libgcc-ng=11.2.0=h1234567_1
libgfortran-ng=7.5.0=ha8ba4b0_17
libgfortran4=7.5.0=ha8ba4b0_17
libgomp=11.2.0=h1234567_1
libpng=1.6.37=hbc83047_0
libstdcxx-ng=11.2.0=h1234567_1
libtiff=4.4.0=hecacb30_2
libwebp=1.2.4=h11a3e52_0
libwebp-base=1.2.4=h5eee18b_0
libxgboost=1.5.0=h6a678d5_2
lz4-c=1.9.3=h295c915_1
mkl=2020.2=256
mkl-service=2.3.0=py38he904b0f_0
mkl_fft=1.3.0=py38h54f3939_0
mkl_random=1.1.1=py38h0573a6f_0
ncurses=6.3=h5eee18b_3
ninja=1.10.2=h06a4308_5
ninja-base=1.10.2=hd09550d_5
numpy=1.18.5=py38ha1c710e_0
numpy-base=1.18.5=py38hde5b4d6_0
openssl=1.1.1s=h7f8727e_0
pandas=1.2.4=py38ha9443f7_0
pillow=9.2.0=py38hace64e9_1
pip=22.2.2=py38h06a4308_0
py-xgboost=1.5.0=py38h06a4308_2
python=3.8.3=hcff3b4d_2
python-dateutil=2.8.2=pyhd3eb1b0_0
pytorch=1.6.0=py3.8_cuda10.2.89_cudnn7.6.5_0
pytz=2022.1=py38h06a4308_0
readline=8.2=h5eee18b_0
scikit-learn=0.23.1=py38h423224d_0
scipy=1.5.2=py38h0b6359f_0
setuptools=65.5.0=py38h06a4308_0
six=1.16.0=pyhd3eb1b0_1
sqlite=3.39.3=h5082296_0
threadpoolctl=2.2.0=pyh0d69192_0
tk=8.6.12=h1ccaba5_0
torchvision=0.7.0=py38_cu102
wheel=0.37.1=pyhd3eb1b0_0
xz=5.2.6=h5eee18b_0
zlib=1.2.13=h5eee18b_0
zstd=1.5.2=ha4553b6_0
```

Put all the below files in the same directory and run 

```shell
python Fed_Unlearn_main.py
```

# `data_preprocess.py`

```python
import torch
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
from torch.utils.data import Dataset,TensorDataset
from torchvision import datasets, transforms
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

"""Function: load data"""
def data_init(FL_params):
    
    kwargs = {'num_workers': 0, 'pin_memory': True} if FL_params.cuda_state else {}
    trainset, testset = data_set(FL_params.data_name)
    test_loader = DataLoader(testset, batch_size=FL_params.test_batch_size, shuffle=True, **kwargs)

    split_index = [int(trainset.__len__()/FL_params.N_total_client)]*(FL_params.N_total_client-1)
    split_index.append(int(trainset.__len__() - int(trainset.__len__()/FL_params.N_total_client)*(FL_params.N_total_client-1)))
    client_dataset = torch.utils.data.random_split(trainset, split_index)

    client_loaders = []
    for ii in range(FL_params.N_total_client):
        client_loaders.append(DataLoader(client_dataset[ii], FL_params.local_batch_size, shuffle=True, **kwargs))

    return client_loaders, test_loader


"""Function: load data"""
def data_init_with_shadow(FL_params):
    
    kwargs = {'num_workers': 0, 'pin_memory': True} if FL_params.cuda_state else {}
    whole_trainset, whole_testset = data_set(FL_params.data_name)
    shadow_split_idx = [int(whole_trainset.__len__()/2), int(whole_trainset.__len__()) -int(whole_trainset.__len__()/2)]
    trainset, shadow_trainset = torch.utils.data.random_split(whole_trainset, shadow_split_idx)
    
    shadow_split_idx = [int(whole_testset.__len__()/2), int(whole_testset.__len__()) -int(whole_testset.__len__()/2)]
    testset, shadow_testset = torch.utils.data.random_split(whole_testset, shadow_split_idx)

    test_loader = DataLoader(testset, batch_size=FL_params.test_batch_size, shuffle=False, **kwargs)
    shadow_test_loader = DataLoader(shadow_testset, batch_size=FL_params.test_batch_size, shuffle=False, **kwargs)

    split_index = [int(trainset.__len__()/FL_params.N_client)]*(FL_params.N_client-1)
    split_index.append(int(trainset.__len__() - int(trainset.__len__()/FL_params.N_client)*(FL_params.N_client-1)))
    client_dataset = torch.utils.data.random_split(trainset, split_index)

    split_index = [int(shadow_trainset.__len__()/FL_params.N_client)]*(FL_params.N_client-1)
    split_index.append(int(shadow_trainset.__len__() - int(shadow_trainset.__len__()/FL_params.N_client)*(FL_params.N_client-1)))
    shadow_client_dataset = torch.utils.data.random_split(shadow_trainset, split_index)
 
    client_loaders = []
    shadow_client_loaders = []
    for ii in range(FL_params.N_client):
        client_loaders.append(DataLoader(client_dataset[ii], FL_params.local_batch_size, shuffle=False, **kwargs))
        shadow_client_loaders.append(DataLoader(shadow_client_dataset[ii], FL_params.local_batch_size, shuffle=False, **kwargs))

    return client_loaders, test_loader, shadow_client_loaders, shadow_test_loader


def data_set(data_name):
    if not data_name in ['mnist','purchase','adult','cifar10']:
        raise TypeError('data_name should be a string, including mnist,purchase,adult,cifar10. ')
    
    # model: 2 conv. layers followed by 2 FC layers
    if(data_name == 'mnist'):
        trainset = datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))

        testset = datasets.MNIST('./data', train=False, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
        
    # model: ResNet-50
    elif(data_name == 'cifar10'):
        transform = transforms.Compose(
            [transforms.ToTensor(),
             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        
        trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
        
        testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    
    # model: 2 FC layers
    elif(data_name == 'purchase'):
        xx = np.load("./data/purchase/purchase_xx.npy")
        yy = np.load("./data/purchase/purchase_y2.npy")
        X_train, X_test, y_train, y_test = train_test_split(xx, yy, test_size=0.2, random_state=42)
        
        X_train_tensor = torch.Tensor(X_train).type(torch.FloatTensor)
        X_test_tensor = torch.Tensor(X_test).type(torch.FloatTensor)
        y_train_tensor = torch.Tensor(y_train).type(torch.LongTensor)
        y_test_tensor = torch.Tensor(y_test).type(torch.LongTensor)
        
        trainset = TensorDataset(X_train_tensor,y_train_tensor)
        testset = TensorDataset(X_test_tensor,y_test_tensor)
        
    # model: 2 FC layers
    elif(data_name == 'adult'):
        #load data
        file_path = "./data/adult/"
        data1 = pd.read_csv(file_path + 'adult.data', header=None)
        data2 = pd.read_csv(file_path + 'adult.test', header=None)
        data2 = data2.replace(' <=50K.', ' <=50K')    
        data2 = data2.replace(' >50K.', ' >50K')
        train_num = data1.shape[0]
        data = pd.concat([data1,data2])
       
        # data transform: str->int
        data = np.array(data, dtype=str)
        labels = data[:,14]
        le= LabelEncoder()
        le.fit(labels)
        labels = le.transform(labels)
        data = data[:,:-1]
        
        categorical_features = [1,3,5,6,7,8,9,13]
        for feature in categorical_features:
            le = LabelEncoder()
            le.fit(data[:, feature])
            data[:, feature] = le.transform(data[:, feature])
        data = data.astype(float)
        
        n_features = data.shape[1]
        numerical_features = list(set(range(n_features)).difference(set(categorical_features)))
        for feature in numerical_features:
            scaler = MinMaxScaler()
            sacled_data = scaler.fit_transform(data[:,feature].reshape(-1,1))
            data[:,feature] = sacled_data.reshape(-1)
        
        # OneHotLabel
        oh_encoder = ColumnTransformer(
            [('oh_enc', OneHotEncoder(sparse=False), categorical_features),], 
            remainder='passthrough' )
        oh_data = oh_encoder.fit_transform(data)
        
        xx = oh_data
        yy = labels
        xx = preprocessing.scale(xx)
        yy = np.array(yy)
        
        xx = torch.Tensor(xx).type(torch.FloatTensor)
        yy = torch.Tensor(yy).type(torch.LongTensor)
        xx_train = xx[0:data1.shape[0],:]
        xx_test = xx[data1.shape[0]:,:]
        yy_train = yy[0:data1.shape[0]]
        yy_test = yy[data1.shape[0]:]

        trainset = TensorDataset(xx_train,yy_train)
        testset = TensorDataset(xx_test,yy_test)
        
    return trainset, testset


"""
Array2Dataset: A class that can transform np.array(tensor matrix) to a torch.Dataset class.  
"""
class Array2Dataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = targets
        self.transform = transform
    def __getitem__(self, index):
        x = self.data[index,:]
        y = self.targets[index]
        return x, y
    def __len__(self):
        return len(self.data)

```

# `Fed_Unlearn_base.py`

```python
import torch
import torch.functional as F
import torch.nn as nn
import torch.optim as optim
import argparse
from torch.utils.data import DataLoader, Dataset
import copy
from sklearn.metrics import accuracy_score
import numpy as np
import time

from model_initiation import model_init
from data_preprocess import data_set

from FL_base import fedavg, global_train_once, FL_Train, FL_Retrain


def federated_learning_unlearning(init_global_model, client_loaders, test_loader, FL_params):
    print(5*"#"+"  Federated Learning Start  "+5*"#")
    std_time = time.time()
    old_GMs, old_CMs = FL_Train(init_global_model, client_loaders, test_loader, FL_params)
    end_time = time.time()
    time_learn = (std_time - end_time)
    print(5*"#"+"  Federated Learning End  "+5*"#")

    print('\n')
    """4.2 unlearning  a client, Federated Unlearning"""
    print(5*"#"+"  Federated Unlearning Start  "+5*"#")
    std_time = time.time()
    FL_params.if_unlearning = True
    FL_params.forget_client_idx = 2
    unlearn_GMs = unlearning(old_GMs, old_CMs, client_loaders, test_loader, FL_params)
    end_time = time.time()
    time_unlearn = (std_time - end_time)
    print(5*"#"+"  Federated Unlearning End  "+5*"#")

    print('\n')
    """4.3 unlearning a client, Federated Unlearning without calibration"""
    print(5*"#"+"  Federated Unlearning without Calibration Start  "+5*"#")
    std_time = time.time()
    uncali_unlearn_GMs = unlearning_without_cali(old_GMs, old_CMs, FL_params)
    end_time = time.time()
    time_unlearn_no_cali = (std_time - end_time)
    print(5*"#"+"  Federated Unlearning without Calibration End  "+5*"#")

    print(" Learning time consuming = {} secods".format(-time_learn))
    print(" Unlearning time consuming = {} secods".format(-time_unlearn)) 
    print(" Unlearning no Cali time consuming = {} secods".format(-time_unlearn_no_cali))

    return old_GMs, unlearn_GMs, uncali_unlearn_GMs, old_CMs


def unlearning(old_GMs, old_CMs, client_data_loaders, test_loader, FL_params):
    """
    Parameters
    ----------
    old_global_models : list of DNN models
        In standard federated learning, all the global models from each round of training are saved.
    old_client_models : list of local client models
        In standard federated learning, the server collects all user models after each round of training.
    client_data_loaders : list of torch.utils.data.DataLoader
        This can be interpreted as each client user's own data, and each Dataloader corresponds to each user's data
    test_loader : torch.utils.data.DataLoader
        The loader for the test set used for testing
    FL_params : Argment()
        The parameter class used to set training parameters

    Returns
    -------
    forget_global_model : One DNN model that has the same structure but different parameters with global_moedel
        DESCRIPTION.
    """

    if(FL_params.if_unlearning == False):
        raise ValueError('FL_params.if_unlearning should be set to True, if you want to unlearning with a certain user')
    if(not(FL_params.forget_client_idx in range(FL_params.N_client))):
        raise ValueError('FL_params.forget_client_idx is note assined correctly, forget_client_idx should in {}'.format(range(FL_params.N_client)))
    if(FL_params.unlearn_interval == 0 or FL_params.unlearn_interval >FL_params.global_epoch):
        raise ValueError('FL_params.unlearn_interval should not be 0, or larger than the number of FL_params.global_epoch')

    old_global_models = copy.deepcopy(old_GMs)
    old_client_models = copy.deepcopy(old_CMs)

    forget_client = FL_params.forget_client_idx
    for ii in range(FL_params.global_epoch):
        temp = old_client_models[ii*FL_params.N_client : ii*FL_params.N_client+FL_params.N_client]
        temp.pop(forget_client)#During Unlearn, the model saved by the forgotten user pops up
        old_client_models.append(temp)
    old_client_models = old_client_models[-FL_params.global_epoch:]

    GM_intv = np.arange(0,FL_params.global_epoch+1, FL_params.unlearn_interval, dtype=np.int16())
    CM_intv  = GM_intv -1
    CM_intv = CM_intv[1:]

    selected_GMs = [old_global_models[ii] for ii in GM_intv]
    selected_CMs = [old_client_models[jj] for jj in CM_intv]

    """1. First, complete the model overlay from the initial model to the first round of global train"""
    """
    Since the inIT_model does not contain any information about the forgotten user at the start of the FL training, you just need to overlay the local Model of the other retained users, You can get the Global Model after the first round of global training.
    """
    epoch = 0
    unlearn_global_models = list()
    unlearn_global_models.append(copy.deepcopy(selected_GMs[0]))
    
    new_global_model = fedavg(selected_CMs[epoch])
    unlearn_global_models.append(copy.deepcopy(new_global_model))
    print("Federated Unlearning Global Epoch  = {}".format(epoch))
    
    """2. Then, the first round of global model as a starting point, the model is gradually corrected"""
    """
    In this step, the global Model obtained from the first round of global training was used as the new starting point for training, and a small amount of training was carried out with the data of the reserved user (a small amount means reducing the local epoch, i.e. Reduce the number of local training rounds for each user. The parameter forget_local_epoch_ratio is to control and reduce the number of local training rounds.) Gets the direction of iteration of the local Model parameter for each reserved user, starting with new_global_model.Note that this part of the user model is ref_client_models.

    Then we use the old_client_models and old_global_models saved from the unforgotten FL training, and the ref_client_models and new_global_Model that we get when we forget a user,To build the global model for the next round


    (ref_client_models - new_global_model) / ||ref_client_models - new_global_model||, Indicates the direction of model parameter iteration starting with a new global model that removes a user.Mark the direction as step_direction

    ||old_client_models - old_global_model||, Indicates the step size of the model parameter iteration starting with the old global model with a user removed.Step step_length

    So, the final direction of the new reference model is step_direction*step_length + new_global_model。
    """
    """
    Intuitive explanation of this part: Usually in IID data, after the data is sharded, the direction of model parameter iteration is roughly the same.The basic idea is to take full advantage of the client-model parameter data saved in standard FL training, and then, by correcting this part of the parameter, apply it to the iteration of the new global model that forgets a user.

    For unforgotten FL: oldGM_t--> oldCM0, oldCM1, oldCM2, oldCM3--> oldGM_t+1
    for unblearning FL: newGM_t-->newCM0, newCM1, newCM2, newCM3--> newGM_t+1
    oldGM_t and newGM_t essentially represents a different starting point for training. However, under the IID data, oldCM and newCM should converge in roughly the same direction.
    Therefore, we get newCM by using newcm-newgm_t as the starting point and training fewer rounds on user data, and then using (newcm-newgm_t)/|| newcm-newgm_t || as the current forgetting setting,
    Direction of model parameter iteration.Take || oldcm-oldgm_t || as the iteration step, and finally use || oldcm-oldgm_t ||*(newcm-newgm_t)/|| newcm-newgm_t |0 |1 for the iteration of the new model.
    FedEraser iterative formula: newGM_t+1 = newGM_t + ||oldCM - oldGM_t||*(newCM - newGM_t)/||newCM - newGM_t||
    """

    CONST_local_epoch = copy.deepcopy(FL_params.local_epoch)
    FL_params.local_epoch = np.ceil(FL_params.local_epoch*FL_params.forget_local_epoch_ratio)
    FL_params.local_epoch = np.int16(FL_params.local_epoch)

    CONST_global_epoch = copy.deepcopy(FL_params.global_epoch)
    FL_params.global_epoch = CM_intv.shape[0]

    print('Local Calibration Training epoch = {}'.format(FL_params.local_epoch))
    for epoch in range(FL_params.global_epoch):
        if(epoch == 0):
            continue
        print("Federated Unlearning Global Epoch  = {}".format(epoch))
        global_model = unlearn_global_models[epoch]

        new_client_models  = global_train_once(global_model, client_data_loaders, test_loader, FL_params)

        new_GM = unlearning_step_once(selected_CMs[epoch], new_client_models, selected_GMs[epoch+1], global_model)
        
        unlearn_global_models.append(new_GM)
    FL_params.local_epoch = CONST_local_epoch
    FL_params.global_epoch = CONST_global_epoch
    return unlearn_global_models
    
def unlearning_step_once(old_client_models, new_client_models, global_model_before_forget, global_model_after_forget):
    """
    Parameters
    ----------
    old_client_models : list of DNN models
        When there is no choice to forget (if_forget=False), use the normal continuous learning training to get each user's local model.The old_client_models do not contain models of users that are forgotten.
        Models that require forgotten users are not discarded in the Forget function
    ref_client_models : list of DNN models
        When choosing to forget (if_forget=True), train with the same Settings as before, except that the local epoch needs to be reduced, other parameters are set in the same way.
        Using the above training Settings, the new global model is taken as the starting point and the reference model is trained.The function of the reference model is to identify the direction of model parameter iteration starting from the new global model
        
    global_model_before_forget : The old global model
        DESCRIPTION.
    global_model_after_forget : The New global model
        DESCRIPTION.

    Returns
    -------
    return_global_model : After one iteration, the new global model under the forgetting setting

    """
    old_param_update = dict() # Model Params： oldCM - oldGM_t
    new_param_update = dict() # Model Params： newCM - newGM_t

    new_global_model_state = global_model_after_forget.state_dict()#newGM_t

    return_model_state = dict() # newGM_t + ||oldCM - oldGM_t||*(newCM - newGM_t)/||newCM - newGM_t||

    assert len(old_client_models) == len(new_client_models)

    for layer in global_model_before_forget.state_dict().keys():
        old_param_update[layer] = 0*global_model_before_forget.state_dict()[layer]
        new_param_update[layer] = 0*global_model_before_forget.state_dict()[layer]

        return_model_state[layer] = 0*global_model_before_forget.state_dict()[layer]

        for ii in range(len(new_client_models)):
            old_param_update[layer] += old_client_models[ii].state_dict()[layer]
            new_param_update[layer] += new_client_models[ii].state_dict()[layer]
        old_param_update[layer] /= (ii+1) # Model Params： oldCM
        new_param_update[layer] /= (ii+1) # Model Params： newCM

        old_param_update[layer] = old_param_update[layer] - global_model_before_forget.state_dict()[layer] # oldCM - oldGM_t
        new_param_update[layer] = new_param_update[layer] - global_model_after_forget.state_dict()[layer] # newCM - newGM_t

        step_length = torch.norm(old_param_update[layer]) #||oldCM - oldGM_t||
        step_direction = new_param_update[layer]/torch.norm(new_param_update[layer]) # (newCM - newGM_t)/||newCM - newGM_t||

        return_model_state[layer] = new_global_model_state[layer] + step_length*step_direction
    
    return_global_model = copy.deepcopy(global_model_after_forget)
    
    return_global_model.load_state_dict(return_model_state)
    
    return return_global_model
    
    
def unlearning_without_cali(old_global_models, old_client_models, FL_params):
    """
    Parameters
    ----------
    old_client_models : list of DNN models
        All user local update models are saved during the federated learning and training process that is not forgotten.
    FL_params : parameters
        All parameters in federated learning and federated forgetting learning

    Returns
    -------
    global_models : List of DNN models
        In each update round, the client model of the user who needs to be forgotten is removed, and the parameters of other users' client models are directly superimposing to form the new Global Model of each round

    """
    """
    The basic process is as follows: For unforgotten FL:oldGM_t--> oldCM0, oldCM1, oldCM2, oldCM3--> oldGM_t+1
                 For unlearning FL: newGM_t-->The parameters of oldCM and oldGM were directly leveraged to update global model--> newGM_t+1
    The update process is as follows: newGM_t+1 = (oldCM - oldGM_t) + newGM_t
    """
    if(FL_params.if_unlearning == False):
        raise ValueError('FL_params.if_unlearning should be set to True, if you want to unlearning with a certain user')

    if(not(FL_params.forget_client_idx in range(FL_params.N_client))):
        raise ValueError('FL_params.forget_client_idx is note assined correctly, forget_client_idx should in {}'.format(range(FL_params.N_client)))
    forget_client = FL_params.forget_client_idx


    for ii in range(FL_params.global_epoch):
        temp = old_client_models[ii*FL_params.N_client : ii*FL_params.N_client+FL_params.N_client]
        temp.pop(forget_client)
        old_client_models.append(temp)
    old_client_models = old_client_models[-FL_params.global_epoch:]

    uncali_global_models = list()
    uncali_global_models.append(copy.deepcopy(old_global_models[0]))
    epoch = 0
    uncali_global_model = fedavg(old_client_models[epoch])
    uncali_global_models.append(copy.deepcopy(uncali_global_model))
    print("Federated Unlearning without Clibration Global Epoch  = {}".format(epoch))

    """
    new_GM_t+1 = newGM_t + (oldCM_t - oldGM_t)

    For standard federated learning:oldGM_t --> oldCM_t --> oldGM_t+1
    For accumulatring:    newGM_t --> (oldCM_t - oldGM_t) --> oldGM_t+1
    For uncalibrated federated forgotten learning, the parameter update of the unforgotten user in standard federated learning is used to directly overlay the new global model to obtain the next round of new global model.
    """
    old_param_update = dict() # (oldCM_t - oldGM_t)
    return_model_state = dict() # newGM_t+1
    
    for epoch in range(FL_params.global_epoch):
        if(epoch == 0):
            continue
        print("Federated Unlearning Global Epoch  = {}".format(epoch))

        current_global_model = uncali_global_models[epoch] # newGM_t
        current_client_models = old_client_models[epoch] # oldCM_t
        old_global_model = old_global_models[epoch] # oldGM_t
        # global_model_before_forget = old_global_models[epoch]#old_GM_t

        for layer in current_global_model.state_dict().keys():
            # State variable initialization
            old_param_update[layer] = 0*current_global_model.state_dict()[layer]
            return_model_state[layer] = 0*current_global_model.state_dict()[layer]
            
            for ii in range(len(current_client_models)):
                old_param_update[layer] += current_client_models[ii].state_dict()[layer]
            old_param_update[layer] /= (ii+1) # oldCM_t
            
            old_param_update[layer] = old_param_update[layer] - old_global_model.state_dict()[layer] # oldCM_t - oldGM_t

            return_model_state[layer] = current_global_model.state_dict()[layer] + old_param_update[layer] # newGM_t + (oldCM_t - oldGM_t)
            
        return_global_model = copy.deepcopy(old_global_models[0])
        return_global_model.load_state_dict(return_model_state)

        uncali_global_models.append(return_global_model)

    return uncali_global_models

```

# `Fed_Unlearn_main.py`

```python
import torch
import torch.functional as F
import torch.nn as nn
import torch.optim as optim
import argparse
from torch.utils.data import DataLoader, Dataset
import copy
from sklearn.metrics import accuracy_score
import numpy as np
import time 

from model_initiation import model_init
from data_preprocess import data_init, data_init_with_shadow
from FL_base import global_train_once
from FL_base import fedavg
from FL_base import test

from FL_base import FL_Train, FL_Retrain
from Fed_Unlearn_base import unlearning, unlearning_without_cali, federated_learning_unlearning
from membership_inference import train_attack_model, attack

"""Step 0. Initialize Federated Unlearning parameters"""
class Arguments():
    def __init__(self):
        # Federated Learning Settings
        self.N_total_client = 100
        self.N_client = 10
        self.data_name = 'mnist' # purchase, cifar10, mnist, adult
        self.global_epoch = 20
        self.local_epoch = 10

        # Model Training Settings
        self.local_batch_size = 64
        self.local_lr = 0.005

        self.test_batch_size = 64
        self.seed = 1
        self.save_all_model = True
        self.cuda_state = torch.cuda.is_available()
        self.use_gpu = True
        self.train_with_test = False
        
        # Federated Unlearning Settings
        self.unlearn_interval = 1 # Used to control how many rounds the model parameters are saved. 1 represents the parameter saved once per round  N_itv in our paper.
        self.forget_client_idx = 2 # If want to forget, change None to the client index
                                # If this parameter is set to False, only the global model after the final training is completed is output
        self.if_retrain = True # If set to True, the global model is retrained using the FL-Retrain function, and data corresponding to the user for the forget_client_IDx number is discarded.
        
        self.if_unlearning = False # If set to False, the global_train_once function will not skip users that need to be forgotten;If set to True, global_train_once skips the forgotten user during training
        
        self.forget_local_epoch_ratio = 0.5 # When a user is selected to be forgotten, other users need to train several rounds of on-line training in their respective data sets to obtain the general direction of model convergence in order to provide the general direction of model convergence.
        # forget_local_epoch_ratio*local_epoch Is the number of rounds of local training when we need to get the convergence direction of each local model
        # self.mia_oldGM = False

def Federated_Unlearning():
    """Step 1.Set the parameters for Federated Unlearning"""
    FL_params = Arguments()
    torch.manual_seed(FL_params.seed)
    # kwargs for data loader 
    print(60*'=')
    print("Step1. Federated Learning Settings \n We use dataset: "+FL_params.data_name+(" for our Federated Unlearning experiment.\n"))

    """Step 2. construct the necessary user private data set required for federated learning, as well as a common test set"""
    print(60 * '=')
    print("Step2. Client data loaded, testing data loaded!!!\n       Initial Model loaded!!!")
    init_global_model = model_init(FL_params.data_name)
    client_all_loaders, test_loader = data_init(FL_params)

    selected_clients=np.random.choice(range(FL_params.N_total_client),size=FL_params.N_client, replace=False)
    client_loaders = list()
    for idx in selected_clients:
        client_loaders.append(client_all_loaders[idx])
    # client_all_loaders = client_loaders[selected_clients]
    # client_loaders, test_loader, shadow_client_loaders, shadow_test_loader = data_init_with_shadow(FL_params)
    """
    This section of the code gets the initialization model init Global Model
    User data loader for FL training Client_loaders and test data loader Test_loader
    User data loader for covert FL training, Shadow_client_loaders, and test data loader Shadowl_test_loader
    """

    """Step 3. Select a client's data to forget, 1.Federated Learning, 2.Unlearning(FedEraser), and 3.(Accumulating)Unlearing without calibration"""
    print(60*'=')
    print("Step3. Fedearated Learning and Unlearning Training...")
    # 
    old_GMs, unlearn_GMs, uncali_unlearn_GMs, old_CMs = federated_learning_unlearning(
        init_global_model, 
        client_loaders, 
        test_loader, 
        FL_params
    )

    if(FL_params.if_retrain == True):
        
        t1 = time.time()
        retrain_GMs = FL_Retrain(init_global_model, client_loaders, test_loader, FL_params)
        t2 = time.time()
        print("Time using = {} seconds".format(t2-t1))

    """Step 4  The member inference attack model is built based on the output of the Target Global Model on client_loaders and test_loaders.In this case, we only do the MIA attack on the model at the end of the training"""
    
    """MIA:Based on the output of oldGM model, MIA attack model was built, and then the attack model was used to attack unlearn GM. If the attack accuracy significantly decreased, it indicated that our unlearn method was indeed effective to remove the user's information"""
    print(60*'=')
    print("Step4. Membership Inference Attack aganist GM...")

    T_epoch = -1
    # MIA setting:Target model == Shadow Model
    old_GM = old_GMs[T_epoch]
    attack_model = train_attack_model(old_GM, client_loaders, test_loader, FL_params)

    print("\nEpoch  = {}".format(T_epoch))
    print("Attacking against FL Standard  ")
    target_model = old_GMs[T_epoch]
    (ACC_old, PRE_old) = attack(target_model, attack_model, client_loaders, test_loader, FL_params)

    if(FL_params.if_retrain == True):
        print("Attacking against FL Retrain  ")
        target_model = retrain_GMs[T_epoch]
        (ACC_retrain, PRE_retrain) = attack(target_model, attack_model, client_loaders, test_loader, FL_params)

    print("Attacking against FL Unlearn  ")
    target_model = unlearn_GMs[T_epoch]
    (ACC_unlearn, PRE_unlearn) = attack(target_model, attack_model, client_loaders, test_loader, FL_params)


if __name__=='__main__':
    Federated_Unlearning()
```

# `FL_base.py`

```python
import torch
import torch.functional as F
import torch.nn as nn
import torch.optim as optim
import argparse
from torch.utils.data import DataLoader, Dataset
import copy
from sklearn.metrics import accuracy_score
import numpy as np
import time

from model_initiation import model_init
from data_preprocess import data_set


def FL_Train(init_global_model, client_data_loaders, test_loader, FL_params):
    all_global_models = list()
    all_client_models = list()
    global_model = init_global_model

    all_global_models.append(copy.deepcopy(global_model))

    for epoch in range(FL_params.global_epoch):
        client_models = global_train_once(global_model, client_data_loaders, test_loader, FL_params)
        # IMPORTANT: It is IMPORTANT to note here that global_train_once is trained directly on the input client_models during training, so the output's client_models are the same set of models as the input's client_models, except that the input is untrained while the output is trained.
        # Therefore, in order to implement Federated Unlearning, we need to save the models in Client -- Models before global Train.You can use DeepCopy, or hard disk IO.
        all_client_models += client_models
        global_model = fedavg(client_models)
        # print(30*'^')
        print("Global Federated Learning epoch = {}".format(epoch))
        # test(global_model, test_loader)
        # print(30*'v')
        # print(len(all_client_models))
        all_global_models.append(copy.deepcopy(global_model))

    return all_global_models, all_client_models


def FL_Retrain(init_global_model, client_data_loaders, test_loader, FL_params):
    if(FL_params.if_retrain == False):
        raise ValueError('FL_params.if_retrain should be set to True, if you want to retrain FL model')
    if(FL_params.forget_client_idx not in range(FL_params.N_client)):
        raise ValueError('FL_params.forget_client_idx should be in [{}], if you want to use standard FL train with forget the certain client dataset.'.format(range(FL_params.N_client)))
    # forget_idx= FL_params.forget_idx
    print('\n')
    print(5*"#"+"  Federated Retraining Start  "+5*"#")
    # std_time = time.time()
    print("Federated Retrain with Forget Client NO.{}".format(FL_params.forget_client_idx))
    retrain_GMs = list()
    all_client_models = list()
    retrain_GMs.append(copy.deepcopy(init_global_model))
    global_model = init_global_model
    for epoch in range(FL_params.global_epoch):
        client_models = global_train_once(global_model, client_data_loaders, test_loader, FL_params)
        # IMPORTANT：It is important to note that global_train_once is trained directly on the input client_models during training, so the output's client_models are the same set of models as the input's client_models, except that the input is untrained while the output is trained.
        #IMPORTANT: Therefore, in order to implement Federated Unlearning, we need to save the models in Client -- Models before global Train.You can use DeepCopy, or hard disk IO.
        global_model = fedavg(client_models)
        print("Global Retraining epoch = {}".format(epoch))
        retrain_GMs.append(copy.deepcopy(global_model))
        
        all_client_models += client_models
    print(5*"#"+"  Federated Retraining End  "+5*"#")
    return retrain_GMs


"""
Function:
For the global round of training, the data and optimizer of each global_ModelT is used. The global model of the previous round is the initial point and the training begins.
NOTE: The global model inputed is the global model for the previous round
      The output client_Models is the model that each user trained separately.
"""
# training sub function
def global_train_once(global_model, client_data_loaders, test_loader, FL_params):
    # Using the model, optimizer, and data of each client, training the initial model with client_models, updating the UPODate -- client_models using the client user's local data and optimizer
    # Note: It is important to Note that global_train_once is only a global update to the parameters of the model
    # update_client_models = list()
    device = torch.device("cuda" if FL_params.use_gpu*FL_params.cuda_state else "cpu")
    device_cpu = torch.device("cpu")

    client_models = []
    client_sgds = []
    for ii in range(FL_params.N_client):
        client_models.append(copy.deepcopy(global_model))
        client_sgds.append(optim.SGD(client_models[ii].parameters(), lr=FL_params.local_lr, momentum=0.9))

    for client_idx in range(FL_params.N_client):
        if(((FL_params.if_retrain) and (FL_params.forget_client_idx == client_idx)) or ((FL_params.if_unlearning) and (FL_params.forget_client_idx == client_idx))):
            continue
        # if((FL_params.if_unlearning) and (FL_params.forget_client_idx == client_idx)):
        #     continue
        # print(30*'-')
        # print("Now training Client No.{}  ".format(client_idx))
        model = client_models[client_idx]
        optimizer = client_sgds[client_idx]

        model.to(device)
        model.train()

        #local training
        for local_epoch in range(FL_params.local_epoch):
            for batch_idx, (data, target) in enumerate(client_data_loaders[client_idx]):
                data = data.to(device)
                target = target.to(device)

                optimizer.zero_grad()
                pred = model(data)
                criteria = nn.CrossEntropyLoss()
                loss = criteria(pred, target)
                loss.backward()
                optimizer.step()

            if(FL_params.train_with_test):
                print("Local Client No. {}, Local Epoch: {}".format(client_idx, local_epoch))
                test(model, test_loader)

        # if(FL_params.use_gpu*FL_params.cuda_state):
        model.to(device_cpu)
        client_models[client_idx] = model

    if(((FL_params.if_retrain) and (FL_params.forget_client_idx == client_idx))):
        #Only retrian needs to discard the Client model;If it's not in Retrain, there's no need to discard the model
        client_models.pop(FL_params.forget_client_idx)
        return client_models
    elif((FL_params.if_unlearning) and (FL_params.forget_client_idx in range(FL_params.N_client))):
        client_models.pop(FL_params.forget_client_idx)
        return client_models
    else:
        return client_models


"""
Function:
Test the performance of the model on the test set
"""
def test(model, test_loader):
    model.eval()
    test_loss = 0
    test_acc = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            criteria = nn.CrossEntropyLoss()
            test_loss += criteria(output, target) # sum up batch loss

            pred = torch.argmax(output,axis=1)
            test_acc += accuracy_score(pred,target)

    test_loss /= len(test_loader.dataset)
    test_acc = test_acc/np.ceil(len(test_loader.dataset)/test_loader.batch_size)
    print('Test set: Average loss: {:.8f}'.format(test_loss))         
    print('Test set: Average acc:  {:.4f}'.format(test_acc))    
    return (test_loss, test_acc)


"""
Function:
FedAvg
"""    
def fedavg(local_models):
# def fedavg(local_models, local_model_weights=None):
    """
    Parameters
    ----------
    local_models : list of local models
        DESCRIPTION. In federated learning, with the global_model as the initial model, each user uses a collection of local models updated with their local data.
    local_model_weights : tensor or array
        DESCRIPTION. The weight of each local model is usually related to the accuracy rate and number of data of the local model.(Bypass)

    Returns
    -------
    update_global_model
        Updated global model using fedavg algorithm
    """
    # N = len(local_models)
    # new_global_model = copy.deepcopy(local_models[0])
    # print(len(local_models))
    global_model = copy.deepcopy(local_models[0])
    avg_state_dict = global_model.state_dict()

    local_state_dicts = list()
    for model in local_models:
        local_state_dicts.append(model.state_dict())

    for layer in avg_state_dict.keys():
        avg_state_dict[layer] *= 0 
        for client_idx in range(len(local_models)):
            avg_state_dict[layer] += local_state_dicts[client_idx][layer]
        avg_state_dict[layer] /= len(local_models)

    global_model.load_state_dict(avg_state_dict)
    return global_model

```

# `membership_inference.py`

```python
import torch
import torch.functional as F
import torch.nn as nn
from torch.nn.functional import softmax
import torch.optim as optim
import argparse
from torch.utils.data import DataLoader, Dataset
import copy
from sklearn.metrics import accuracy_score
import numpy as np
from model_initiation import model_init
from data_preprocess import data_set
from FL_base import global_train_once
from FL_base import fedavg
from FL_base import test
from sklearn.linear_model import LogisticRegression
from FL_base import FL_Train, FL_Retrain
from Fed_Unlearn_base import unlearning, unlearning_without_cali, federated_learning_unlearning

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split


def attack(target_model, attack_model, client_loaders, test_loader, FL_params):
    n_class_dict = dict()
    n_class_dict['adult'] = 2
    n_class_dict['purchase'] = 2
    n_class_dict['mnist'] = 10
    n_class_dict['cifar10'] = 10
    
    N_class = n_class_dict[FL_params.data_name]
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    target_model.to(device)

    target_model.eval()

    # The predictive output of forgotten user data after passing through the target model.
    unlearn_X = torch.zeros([1,N_class])
    unlearn_X = unlearn_X.to(device)
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(client_loaders[FL_params.forget_client_idx]):
            data = data.to(device)
            out = target_model(data)
            unlearn_X = torch.cat([unlearn_X, out])

    unlearn_X = unlearn_X[1:,:]
    unlearn_X = softmax(unlearn_X,dim = 1)
    unlearn_X = unlearn_X.cpu().detach().numpy()

    unlearn_X.sort(axis=1)
    unlearn_y = np.ones(unlearn_X.shape[0])
    unlearn_y = unlearn_y.astype(np.int16)

    N_unlearn_sample = len(unlearn_y)

    # Test data, predictive output obtained after passing the target model
    test_X = torch.zeros([1, N_class])
    test_X = test_X.to(device)
    with torch.no_grad():
        for _, (data, target) in enumerate(test_loader):
            data = data.to(device)
            out = target_model(data)
            test_X = torch.cat([test_X, out])
            
            if(test_X.shape[0] > N_unlearn_sample):
                break
    test_X = test_X[1:N_unlearn_sample+1,:]
    test_X = softmax(test_X,dim = 1)
    test_X = test_X.cpu().detach().numpy()

    test_X.sort(axis=1)
    test_y = np.zeros(test_X.shape[0])
    test_y = test_y.astype(np.int16)

    # The data of the forgotten user passed through the output of the target model, and the data of the test set passed through the output of the target model were spliced together
    #The balanced data set that forms the 50% train 50% test.
    XX = np.vstack((unlearn_X, test_X))
    YY = np.hstack((unlearn_y, test_y))

    pred_YY = attack_model.predict(XX)
    pre = precision_score(YY, pred_YY, pos_label=1)
    rec = recall_score(YY, pred_YY, pos_label=1)
    print("MIA Attacker precision = {:.4f}".format(pre))
    print("MIA Attacker recall = {:.4f}".format(rec))
    
    return (pre, rec)


def train_attack_model(shadow_old_GM, shadow_client_loaders, shadow_test_loader, FL_params):
    shadow_model = shadow_old_GM
    n_class_dict = dict()
    n_class_dict['adult'] = 2
    n_class_dict['purchase'] = 2
    n_class_dict['mnist'] = 10
    n_class_dict['cifar10'] = 10

    N_class = n_class_dict[FL_params.data_name]

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    shadow_model.to(device)

    shadow_model.eval()
    pred_4_mem = torch.zeros([1,N_class])
    pred_4_mem = pred_4_mem.to(device)
    with torch.no_grad():
        for ii in range(len(shadow_client_loaders)):
            data_loader = shadow_client_loaders[ii]
            
            for batch_idx, (data, target) in enumerate(data_loader):
                    data = data.to(device)
                    out = shadow_model(data)
                    pred_4_mem = torch.cat([pred_4_mem, out])
    pred_4_mem = pred_4_mem[1:,:]
    pred_4_mem = softmax(pred_4_mem,dim = 1)
    pred_4_mem = pred_4_mem.cpu()
    pred_4_mem = pred_4_mem.detach().numpy()

    ####
    pred_4_nonmem = torch.zeros([1,N_class])
    pred_4_nonmem = pred_4_nonmem.to(device)
    with torch.no_grad():
        for batch, (data, target) in enumerate(shadow_test_loader):
            data = data.to(device)
            out = shadow_model(data)
            pred_4_nonmem = torch.cat([pred_4_nonmem, out])
    pred_4_nonmem = pred_4_nonmem[1:,:]
    pred_4_nonmem = softmax(pred_4_nonmem,dim = 1)
    pred_4_nonmem = pred_4_nonmem.cpu()
    pred_4_nonmem = pred_4_nonmem.detach().numpy()

    att_y = np.hstack((np.ones(pred_4_mem.shape[0]), np.zeros(pred_4_nonmem.shape[0])))
    att_y = att_y.astype(np.int16)
    
    att_X = np.vstack((pred_4_mem, pred_4_nonmem))
    att_X.sort(axis=1)
    
    X_train,X_test, y_train, y_test = train_test_split(att_X, att_y, test_size = 0.1)
    
    attacker = XGBClassifier(n_estimators = 300,
                              n_jobs = -1,
                                max_depth = 30,
                              objective = 'binary:logistic',
                              booster="gbtree",
                               scale_pos_weight = pred_4_nonmem.shape[0]/pred_4_mem.shape[0]
                              )

    attacker.fit(X_train, y_train)

    return attacker

```

# `model_initiation.py`

```python
"""
Adapted from FedEraser code
"""
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


def model_init(data_name):
    if(data_name == 'mnist'):
        model = Net_mnist()
    elif(data_name == 'cifar10'):
        model = Net_cifar10()
    elif(data_name == 'purchase'):
        model = Net_purchase()
    elif(data_name == 'adult'):
        model = Net_adult()
    return model


class Net_mnist(nn.Module):
    def __init__(self):
        super(Net_mnist, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


class Net_purchase(nn.Module):
    def __init__(self):
        super(Net_purchase, self).__init__()
        self.fc1 = nn.Linear(600, 300)
        self.fc2 = nn.Linear(300, 50)
        self.fc3 = nn.Linear(50,2)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x


class Net_adult(nn.Module):
    def __init__(self):
        super(Net_adult, self).__init__()
        self.fc1 = nn.Linear(108, 50)
        self.fc2 = nn.Linear(50, 10)
        self.fc3 = nn.Linear(10,2)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x     


class Net_cifar10(nn.Module):
    def __init__(self):
        super(Net_cifar10, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


class All_CNN(nn.Module):
    def __init__(self, filters_percentage=1., n_channels=3, num_classes=10, dropout=False, batch_norm=True):
        super(All_CNN, self).__init__()
        n_filter1 = int(96 * filters_percentage)
        n_filter2 = int(192 * filters_percentage)
        self.features = nn.Sequential(
            Conv(n_channels, n_filter1, kernel_size=3, batch_norm=batch_norm),
            Conv(n_filter1, n_filter1, kernel_size=3, batch_norm=batch_norm),
            Conv(n_filter1, n_filter2, kernel_size=3, stride=2, padding=1, batch_norm=batch_norm),
            nn.Dropout(inplace=True) if dropout else Identity(),
            Conv(n_filter2, n_filter2, kernel_size=3, stride=1, batch_norm=batch_norm),
            Conv(n_filter2, n_filter2, kernel_size=3, stride=1, batch_norm=batch_norm),
            Conv(n_filter2, n_filter2, kernel_size=3, stride=2, padding=1, batch_norm=batch_norm),  # 14
            nn.Dropout(inplace=True) if dropout else Identity(),
            Conv(n_filter2, n_filter2, kernel_size=3, stride=1, batch_norm=batch_norm),
            Conv(n_filter2, n_filter2, kernel_size=1, stride=1, batch_norm=batch_norm),
            nn.AvgPool2d(8),
            Flatten(),
        )
        self.classifier = nn.Sequential(
            nn.Linear(n_filter2, num_classes)
        )

    def forward(self, x):
        features = self.features(x)
        output = self.classifier(features)
        return output  


class Conv(nn.Sequential):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=None, output_padding=0,
                 activation_fn=nn.ReLU, batch_norm=True, transpose=False):
        if padding is None:
            padding = (kernel_size - 1) // 2
        model = []
        if not transpose:
            model += [nn.Conv2d(
                in_channels, 
                out_channels, 
                kernel_size=kernel_size, 
                stride=stride, 
                padding=padding,
                bias=not batch_norm
            )]
        else:
            model += [nn.ConvTranspose2d(
                in_channels, 
                out_channels, 
                kernel_size, 
                stride=stride, 
                padding=padding,
                output_padding=output_padding, 
                bias=not batch_norm
            )]
        if batch_norm:
            model += [nn.BatchNorm2d(out_channels, affine=True)]
        model += [activation_fn()]
        super(Conv, self).__init__(*model) 


class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x  


class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()
    def forward(self,x):
        return x.view(x.size(0), -1)
```

# References

- G. Liu, X. Ma, Y. Yang, C. Wang, and J. Liu, “FedEraser: Enabling Efficient Client-Level Data Removal from Federated Learning Models,” in 2021 IEEE/ACM 29th International Symposium on Quality of Service (IWQOS), 2021, pp. 1–10. [[Paper](https://ieeexplore.ieee.org/abstract/document/9521274)]